<a href="https://colab.research.google.com/github/chi-saku/comfyui-google-colab-starter/blob/main/forge_complete_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎨 Forge環境構築（ワンクリック版）
#@markdown 📖 講座と同じForge環境を自動構築します。

print("🚀 Forge環境構築を開始します...")
print("📖 講座で使用している環境と完全に同じものを構築中...")

from google.colab import drive
import os, pathlib, shutil, subprocess

print("\n📁 Google Drive をマウント中...")
drive.mount('/content/drive')
print("✅ Google Drive接続完了")


In [ ]:
#@title 🔧 Forge本体のセットアップ
print("🔧 Forge本体をセットアップ中...")

# フォルダのパスを定義
forge_dir = pathlib.Path('/content/drive/MyDrive/AI/Forge')
repo_dir = forge_dir / 'forge-webui'

# 既存のForge本体を削除（クリーンインストール）
if repo_dir.exists():
    print("  🔄 古いForge本体をリセットします...")
    shutil.rmtree(repo_dir, ignore_errors=True)
    print("  ✅ 古いForge本体の削除完了")
else:
    print("  📦 Forge本体を新規インストールします...")

# 最新版のForgeをダウンロード
print("  ⬇️ 最新のForgeをダウンロード中...")
try:
    subprocess.run([
        'git', 'clone', '--depth', '1',
        'https://github.com/lllyasviel/stable-diffusion-webui-forge.git',
        str(repo_dir)
    ], check=True, capture_output=True, text=True)
    print("  ✅ Forge本体のダウンロード完了")
except subprocess.CalledProcessError as e:
    print(f"  ❌ Forgeダウンロード失敗: {e}")
    print("  💡 ネットワーク接続を確認してリトライしてください")
    raise


In [ ]:
#@title 🔌 ControlNet拡張の自動インストール
print("🔌 ControlNet拡張を自動インストール中...")

ext_dir = repo_dir / 'extensions'
ext_dir.mkdir(parents=True, exist_ok=True)

# ControlNet拡張をインストール
controlnet_dir = ext_dir / 'sd-webui-controlnet'
if not controlnet_dir.exists():
    print("  📦 ControlNet拡張をインストール中...")
    try:
        subprocess.run([
            'git', 'clone', '--depth', '1',
            'https://github.com/Mikubill/sd-webui-controlnet.git',
            str(controlnet_dir)
        ], check=True)
        print("  ✅ ControlNet拡張インストール完了")
    except subprocess.CalledProcessError as e:
        print(f"  ❌ ControlNet拡張インストール失敗: {e}")
        raise
else:
    print("  ✅ ControlNet拡張は既に存在します")


In [ ]:
#@title 🔗 モデル・出力フォルダの連携設定
print("🔗 フォルダの連携を設定中...")

# リンクするフォルダの対応リスト
link_defs = {
    repo_dir / 'models' / 'Stable-diffusion': '/content/drive/MyDrive/AI/models/Stable-diffusion',
    repo_dir / 'models' / 'VAE': '/content/drive/MyDrive/AI/models/VAE',
    repo_dir / 'models' / 'Lora': '/content/drive/MyDrive/AI/models/Lora',
    repo_dir / 'models' / 'ControlNet': '/content/drive/MyDrive/AI/models/ControlNet',
    repo_dir / 'embeddings': '/content/drive/MyDrive/AI/models/embeddings',
    repo_dir / 'extensions': '/content/drive/MyDrive/AI/Forge_Extensions',
    repo_dir / 'outputs': '/content/drive/MyDrive/AI/Forge_Outputs',
}

for link_path, real_path_str in link_defs.items():
    real_path = pathlib.Path(real_path_str)

    # Google Drive側に実体フォルダを作成
    real_path.mkdir(parents=True, exist_ok=True)

    # 既存フォルダを削除してシンボリックリンクに置き換え
    if link_path.exists():
        print(f"  🧹 既存の '{link_path.name}' を掃除中...")
        if link_path.is_symlink():
            link_path.unlink()
        else:
            shutil.rmtree(link_path, ignore_errors=True)

    # シンボリックリンクを作成
    os.symlink(real_path, link_path, target_is_directory=True)
    print(f"  ✅ '{link_path.name}' の連携完了")

print('🎉 全てのフォルダ連携が完了しました！')


In [ ]:
#@title 🎯 ControlNetモデルの自動ダウンロード
download_controlnet = True #@param {type:"boolean"}

if download_controlnet:
    print("🎯 講座でよく使うControlNetモデルをダウンロード中...")

    controlnet_path = pathlib.Path('/content/drive/MyDrive/AI/models/ControlNet')

    # 講座で使用する基本3種
    models = {
        "control_v11p_sd15_canny.safetensors": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.safetensors",
        "control_v11p_sd15_openpose.safetensors": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.safetensors",
        "control_v11f1p_sd15_depth.safetensors": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.safetensors"
    }

    for filename, url in models.items():
        filepath = controlnet_path / filename
        if not filepath.exists():
            print(f"  📥 {filename} をダウンロード中...")
            try:
                subprocess.run(['wget', '-q', '-O', str(filepath), url], check=True)
                print(f"  ✅ {filename} ダウンロード完了")
            except subprocess.CalledProcessError:
                print(f"  ❌ {filename} ダウンロード失敗")
                print(f"     手動ダウンロードURL: {url}")
        else:
            print(f"  ✅ {filename} は既存")

    print("🎯 ControlNetモデルの準備完了！")
else:
    print("⏭️ ControlNetモデルのダウンロードをスキップしました")


In [ ]:
#@title 📋 環境情報の表示（トラブルシューティング用）
import sys, torch

print("📋 環境情報（問題があれば講師に送信してください）:")
print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPUが利用できません")

# モデル確認
models_dir = pathlib.Path('/content/drive/MyDrive/AI/models/Stable-diffusion')
if models_dir.exists():
    models = list(models_dir.glob('*.safetensors')) + list(models_dir.glob('*.ckpt'))
    print(f"利用可能なモデル数: {len(models)}")
    if models:
        print("最初の3つ:")
        for model in models[:3]:
            print(f"  - {model.name}")


In [ ]:
#@title 📋 環境情報の表示（トラブルシューティング用）
import sys, torch

print("📋 環境情報（問題があれば講師に送信してください）:")
print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPUが利用できません")

# モデル確認
models_dir = pathlib.Path('/content/drive/MyDrive/AI/models/Stable-diffusion')
if models_dir.exists():
    models = list(models_dir.glob('*.safetensors')) + list(models_dir.glob('*.ckpt'))
    print(f"利用可能なモデル数: {len(models)}")
    if models:
        print("最初の3つ:")
        for model in models[:3]:
            print(f"  - {model.name}")
